In [1]:
%matplotlib inline

## Action Plan
1. Create Validation and Sample sets
2. Rearrange image files into their respective directories 
3. Finetune and Train model
4. Generate predictions
5. Validate predictions
6. Submit predictions to Kaggle

In [2]:
import os, sys
#current_dir = os.getcwd()
DATA_HOME_DIR =  '/home/denys/projects/ml-models/kaggle/dog-breed-identification/data/'

## Create validation set and sample

In [7]:
#Create directories
%cd $DATA_HOME_DIR
%mkdir results
%mkdir models
%mkdir valid
%mkdir -p sample/train
%mkdir -p sample/test
%mkdir -p sample/valid
%mkdir -p sample/results
%mkdir -p test/unknown

/home/denys/projects/ml-models/kaggle/dog-breed-identification/data
mkdir: cannot create directory ‘results’: File exists
mkdir: cannot create directory ‘models’: File exists
mkdir: cannot create directory ‘valid’: File exists


In [3]:
from glob import glob
import numpy as np
import pandas as pd

In [4]:
%cd $DATA_HOME_DIR/train

/home/denys/projects/ml-models/kaggle/dog-breed-identification/data/train


In [10]:
VALID_DATA_LEN = 2000

g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(VALID_DATA_LEN): os.rename(shuf[i], DATA_HOME_DIR + '/valid/' + shuf[i])

In [57]:
%cd $DATA_HOME_DIR/train

VALID_DATA_LEN = 2000

labels_df = pd.read_csv("../data/labels.csv")

def get_breed(id):
    print id
    print labels_df.loc[labels_df["id"] == id]["breed"]
    return labels_df.loc[labels_df["id"] == id]["breed"].values[0]

/home/denys/projects/ml-models/kaggle/dog-breed-identification/data/train


In [71]:
def prepare_data_structure(dir):
    train_path = DATA_HOME_DIR + dir
    g = glob(train_path + '*.jpg')
    for i in range(len(g)):
        file_name = g[i][-36:]
        dir_path = train_path + get_breed(file_name[:-4])
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
        os.rename(train_path + "/" + file_name, dir_path + "/" + file_name)

In [73]:
prepare_data_structure("train/")
prepare_data_structure("valid/")

Prepare sample data

In [8]:
from shutil import copyfile

In [27]:
def prepare_sample_data_structure(dir):
    g = glob('*/*.jpg')
    shuf = np.random.permutation(g)
    for i in range(200):
        dir_path = DATA_HOME_DIR + "sample/" + dir + shuf[i][:-37];
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
        copyfile(shuf[i], DATA_HOME_DIR + 'sample/' + dir + shuf[i])

In [10]:
%cd $DATA_HOME_DIR/train

/home/denys/projects/ml-models/kaggle/dog-breed-identification/data/train


In [ ]:
prepare_sample_data_structure("train/")

In [25]:
%cd $DATA_HOME_DIR/valid

/home/denys/projects/ml-models/kaggle/dog-breed-identification/data/valid


In [28]:
prepare_sample_data_structure("valid/")

Prepare test data

In [75]:
%cd $DATA_HOME_DIR/test

/home/denys/projects/ml-models/kaggle/dog-breed-identification/data/test


In [77]:
g = glob('*.jpg')
for i in range(len(g)):
    os.rename(g[i], "unknown/" + g[i])

In [4]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [29]:
from sklearn.preprocessing import OneHotEncoder

from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Dense
from keras.models import Model

In [77]:
def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, batch_size=4, class_mode='categorical',
                target_size=(224,224)):
    return gen.flow_from_directory(dirname, target_size=target_size, 
                                   class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

def get_data(path):
    batches = get_batches(path, shuffle=False, batch_size=1, class_mode=None)
    return np.concatenate([batches.next() for i in range(batches.samples)])

In [31]:
#val_sample_data = get_data(DATA_HOME_DIR+'/sample/valid')

Found 200 images belonging to 95 classes.


In [35]:
bt = get_batches(DATA_HOME_DIR+'/sample/valid', shuffle=False, batch_size=1, class_mode=None)

Found 200 images belonging to 95 classes.


In [41]:
# list of properies bt.__dict__

In [78]:
val_data = get_data(DATA_HOME_DIR+'valid')

Found 2000 images belonging to 120 classes.


In [79]:
train_data = get_data(DATA_HOME_DIR + 'train')

Found 8222 images belonging to 120 classes.


In [43]:
model_path = DATA_HOME_DIR + "models/"

In [80]:
save_array(model_path + 'train_data.bc', train_data)
save_array(model_path + 'valid_data.bc', val_data)

In [65]:
train_data = load_array(model_path+'train_data.bc')
val_data = load_array(model_path+'valid_data.bc')

In [81]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [82]:
# Use batch size of 1 since we're just doing preprocessing on the CPU
val_batches = get_batches(DATA_HOME_DIR + 'valid', shuffle=False, batch_size=1)
batches = get_batches(DATA_HOME_DIR + 'train', shuffle=False, batch_size=1)

Found 2000 images belonging to 120 classes.
Found 8222 images belonging to 120 classes.


In [83]:
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
train_labels = onehot(trn_classes)
val_filenames = val_batches.filenames
train_filenames = batches.filenames


In [104]:
 val_class_indices = val_batches.class_indices

In [13]:
train_data.shape

(8222, 224, 224, 3)

In [84]:
model_vgg16 = VGG16(weights='imagenet', include_top=False)

In [15]:
model_vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [16]:
model_vgg16.output

import keras; print(keras.__version__)

2.0.9


In [85]:
from keras.layers import Dense, GlobalAveragePooling2D
# add a global spatial average pooling layer
x = model_vgg16.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer
predictions = Dense(120, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=model_vgg16.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in model_vgg16.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [86]:
batch_size=20

gen=image.ImageDataGenerator()
batches = gen.flow(train_data, train_labels, batch_size=batch_size, shuffle=True)
val_batches = gen.flow(val_data, val_labels, batch_size=batch_size, shuffle=False)

We'll define a simple function for fitting models, just to save a little typing...

In [ ]:
def fit_model(model, batches, val_batches, nb_epoch=1):
    model.fit_generator(batches, steps_per_epoch=batches.n / batch_size, epochs=nb_epoch, 
                        validation_data=val_batches, validation_steps=val_batches.n, verbose=1)

In [ ]:
fit_model(model, batches, val_batches, nb_epoch=2)

Epoch 1/2
328/411 [======================>.......] - ETA: 18s - loss: 12.9160

In [57]:
model.save_weights(model_path + "vgg16_2epoch.weights")

In [ ]:
probs = model.predict_generator(val_batches, steps=val_batches.n / batch_size)

In [59]:
def plots_idx(idx, titles=None):
    plots([image.load_img(path + 'valid/' + filenames[i]) for i in idx], titles=titles)

In [63]:
probs[9]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  1.], dtype=float32)

In [55]:
val_filenames = val_batches.filenames
train_filenames = batches.filenames

In [67]:
#1. A few correct labels at random
correct = np.where(preds==val_labels[:,1])[0]
idx = permutation(correct)[:n_view]
plots_idx(idx, probs[idx])

2

In [ ]:

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(...)